In [1]:
!pip install icecream

# Key Advantages of IceCream Over Traditional `print()`

The `icecream` library (`ic`) is a powerful alternative to `print()` for debugging and understanding code flow. Its standout features include:

## 1. Less Typing
- Automatically prints variable names and values
  Example: `ic(x)` → `ic| x: 5`
- No need to write manual format strings like `print(f"x: {x}")`

## 2. Toggle On/Off with a Flag
- Globally disable all output: `ic.disable()`
- Enable when needed: `ic.enable()`
- No more commenting out/deleting `print()` statements

## 3. Customizable Output
Modify output format with:
```python
# Add timestamps
ic.configureOutput(prefix=lambda: f"{datetime.now()} | ")

# Include file/line context
ic.configureOutput(prefix=lambda: f"{os.path.basename(__file__)} | ")

In [2]:
from icecream import ic

### Getting Started with IceCream (`ic`)


In [3]:
a = 10
b = ["10", 20, 30, "a", "b"]
c = {"a": 10, "b": 20, "c": 30, "d": 40}
ic(a)
ic(b)
ic(c)
print()

ic| a: 10
ic| b: ['10', 20, 30, 'a', 'b']
ic| c: {'a': 10, 'b': 20, 'c': 30, 'd': 40}


### IceCream's Return Value Behavior

`ic()` has a useful feature: **it returns its input value(s)** while also printing them. This allows for seamless debugging without breaking existing code flow.

In [4]:
def add(a, b):
    return a + b


for i in range(3):
    i = ic(i)
    ic(add(i, i - 1))

ic| i: 0
ic| add(i, i-1): -1
ic| i: 1
ic| add(i, i-1): 1
ic| i: 2
ic| add(i, i-1): 3


### IceCream Behavior Without Variables

When `ic()` is called without any arguments:

- It prints the **filename**, **line number**, and **parent function** where it was called
- In Jupyter notebooks, it shows the **cell line number** instead of the module line number
- This is useful for tracing execution flow

In [5]:
import random

min_ = 100
max_ = 200
mean = (min_ + max_) / 2
sample_size = 10
below_or_at_mean = []
above_mean = []

random.seed(0)
for i in range(sample_size):
    num = random.randint(min_, max_)
    ic(num)
    if num <= mean:
        ic()
        below_or_at_mean.append(num)
    else:
        ic()
        above_mean.append(num)

ic| num: 149
ic| 3303523034.py:15 in <module> at 22:07:51.406
ic| num: 197
ic| 3303523034.py:18 in <module> at 22:07:51.413
ic| num: 153
ic| 3303523034.py:18 in <module> at 22:07:51.420
ic| num: 105
ic| 3303523034.py:15 in <module> at 22:07:51.426
ic| num: 133
ic| 3303523034.py:15 in <module> at 22:07:51.432
ic| num: 165
ic| 3303523034.py:18 in <module> at 22:07:51.438
ic| num: 162
ic| 3303523034.py:18 in <module> at 22:07:51.445
ic| num: 151
ic| 3303523034.py:18 in <module> at 22:07:51.451
ic| num: 200
ic| 3303523034.py:18 in <module> at 22:07:51.457
ic| num: 138
ic| 3303523034.py:15 in <module> at 22:07:51.464


### Customize the output

In [6]:
# Option 1
ic.configureOutput(
    prefix="debug| ",
)
a = 100
ic(a)

debug| a: 100


100

In [7]:
from datetime import datetime
import importlib

now = lambda: datetime.now().strftime("%H:%M:%S")
ic.configureOutput(
    prefix=f"{now()} |",
)
a = 100
ic(a)

module1 = importlib.import_module("package1.module1")

get_file = lambda module: module.__name__
ic.configureOutput(
    prefix=f"{get_file(module1)} |",
)
a = 100
ic(a)

22:07:51 |a: 100
package1.module1 |a: 100


100

### Advanced

# How IceCream Uses `__repr__` for Display

Demonstrating that IceCream uses Python's `__repr__` under the hood for object display:



### Standard Behavior (Without Custom `__repr__`)

In [8]:
import json

# json_data = json.dumps({"a": 1, "b": 2, "c": 3}, indent=4)

json_data = """"{
                  "a": 1,
                  "b": 2,
                  "c": 3
                }"""
ic(json_data)
print()

package1.module1 |json_data: ('"{
                             '
                              '                  "a": 1,
                             '
                              '                  "b": 2,
                             '
                              '                  "c": 3
                             '
                              '                }')


### Standard Behavior (Custom `__repr__`)

#### Option 1:

In [9]:
from icecream import argumentToString

# Your JSON data (fixed the triple quotes)
json_data = """{
    "a": 1,
    "b": 2,
    "c": 3
}"""


class JSON(str):
    pass


# under the hood argumentToString is return from functools.singledispatch
# Register custom formatter for JSON class
@argumentToString.register
def _(data: JSON):
    # Parse and pretty-print the JSON
    parsed = json.loads(data)
    return json.dumps(parsed, indent=2)


# Test the output
ic(JSON(json_data))
print()

package1.module1 |JSON(json_data): {
                                     "a": 1,
                                     "b": 2,
                                     "c": 3
                                   }


### Option 2:

In [10]:
# Your JSON data (fixed the triple quotes)
json_data = """{
    "a": 1,
    "b": 2,
    "c": 3
}"""


class JSON(str):  # Inheritance only for __new__ method
    """Wrapper class for JSON strings"""

    def __repr__(self):
        # Parse and pretty-print the JSON
        parsed = json.loads(self)
        return json.dumps(parsed, indent=2)


ic(JSON(json_data))
print()

package1.module1 |JSON(json_data): {
                                     "a": 1,
                                     "b": 2,
                                     "c": 3
                                   }
